# Cleaning the data together

In [1]:

import requests
import calendar
import time
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# this setting widens how many characters pandas will display in a column:
pd.options.display.max_colwidth = 400

from sklearn.feature_extraction.text import TfidfVectorizer
import re

Import both datasets

In [2]:
vaush_df = pd.read_csv('./data/vaush.csv')
destiny_df = pd.read_csv('./data/destiny.csv')

In [3]:
vd_df =pd.concat([vaush_df,destiny_df])

In [4]:
vd_df = vd_df.drop('Unnamed: 0', axis=1)

In [5]:
vd_df

,subreddit,selftext,title,created_utc,full_text
0,VaushV,"Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?",Anyone Noticing Vaush's Memory Issues Lately?,1668472458,"Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?"
1,VaushV,NaN,Would be a shame if this unofficial site received a lot of bad data,1668471465,Would be a shame if this unofficial site received a lot of bad data
2,VaushV,NaN,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,1668471209,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.
3,VaushV,"Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)",How do you feel about the global population reaching 8 billion?,1668470313,"How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)"
4,VaushV,"Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trump being held accountable or Stopping the rise o...",Hot take: Trump SHOULDN’T be arrested/indicted,1668468418,"Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trum..."
...,...,...,...,...,...
4995,Destiny,NaN,Fine day isn’t it fellow factorio enjoyers,1673121545,Fine day isn’t it fellow factorio enjoyers
4996,Destiny,[removed],"How Destiny is damaging the autistic community, by using autism as an insult.",1673121497,"How Destiny is damaging the autistic community, by using autism as an insult. [removed]"
4997,Destiny,NaN,Has destiny ever talked about how fucking anti semitic badempanda is. He claimed shit like Jews were European and not “indigenous” wild asf,1673121490,Has destiny ever talked about how fucking anti semitic badempanda is. He claimed shit like Jews were European and not “indigenous” wild asf
4998,Destiny,"[This is the clip I am referencing.](https://www.reddit.com/r/SouthJersey/comments/104zjce/who_should_yield/) \n\nDestiny watched this clip on stream and discussed who should have yielded. I believe he ended on the guy in the white car after some deliberation, which is 100% correct. He had a yield sign that is on the other side of the light post in the middle of the turn that is hard to see be...",The driver in the white car from that clip should have yielded (proof),1673121004,"The driver in the white car from that clip should have yielded (proof) [This is the clip I am referencing.](https://www.reddit.com/r/SouthJersey/comments/104zjce/who_should_yield/) \n\nDestiny watched this clip on stream and discussed who should have yielded. I believe he ended on the guy in the white car after some deliberation, which is 100% correct. He had a yield sign that is on the other ..."


## copy df to apply regex pattern

In [6]:
pattern = re.compile('\[(.*?)\]\(.*?\)')

In [7]:
vaush_destiny_df = vd_df.copy()
vaush_destiny_df['no_links'] = vd_df['full_text'].apply(lambda x: pattern.sub(r'\1',str(x)))

In [8]:
vaush_destiny_df['no_links_2'] = vaush_destiny_df['no_links'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'http:\S+', '', regex=True).replace(r'&amp\S+','', regex = True).replace(r'\n','',regex = True).replace(r'\[removed\]','', regex=True).replace(r'\[deleted\]','', regex=True )



In [9]:
vaush_destiny_df

,subreddit,selftext,title,created_utc,full_text,no_links,no_links_2
0,VaushV,"Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?",Anyone Noticing Vaush's Memory Issues Lately?,1668472458,"Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?"
1,VaushV,NaN,Would be a shame if this unofficial site received a lot of bad data,1668471465,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data
2,VaushV,NaN,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,1668471209,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.
3,VaushV,"Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)",How do you feel about the global population reaching 8 billion?,1668470313,"How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\nView Poll","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. View Poll"
4,VaushV,"Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trump being held accountable or Stopping the rise o...",Hot take: Trump SHOULDN’T be arrested/indicted,1668468418,"Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trum...","Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republica

In [10]:
vaush_destiny_df.to_csv('./data/vaush_destiny.csv')

In [11]:
cvec = CountVectorizer(stop_words='english',ngram_range = (1, 2))

In [12]:
cvec.fit(vaush_destiny_df['no_links_2'])

CountVectorizer(ngram_range=(1, 2), stop_words='english')

In [13]:
vec_df = cvec.transform(vaush_destiny_df['no_links_2'])

In [14]:
vec_df = pd.DataFrame(vec_df.todense(), columns = cvec.get_feature_names_out())

In [15]:
vec_df.sum().sort_values(ascending=False).head(50)

people       2898
like         2470
just         2312
destiny      2296
think        1588
don          1405
right        1113
vaush        1048
know         1042
time          904
really        851
good          788
make          735
want          714
men           712
way           705
say           701
does          689
ve            677
going         624
art           597
said          584
left          580
new           575
twitter       575
poll          572
bad           565
did           559
women         556
actually      532
feel          523
need          519
view          518
shit          511
person        508
doesn         508
things        501
lot           489
video         486
ai            471
saying        469
thing         464
stream        460
got           456
point         450
view poll     426
isn           411
doing         389
post          388
talking       380
dtype: int64

Combine df and vecorized df

In [16]:
vaush_destiny_df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998, 4999],
           dtype='int64', length=9934)

In [17]:
vaush_destiny_df.duplicated().sum()

0

In [18]:
vaush_destiny_df = vaush_destiny_df.reset_index()

In [19]:
vaush_destiny_df

,index,subreddit,selftext,title,created_utc,full_text,no_links,no_links_2
0,0,VaushV,"Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?",Anyone Noticing Vaush's Memory Issues Lately?,1668472458,"Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?"
1,1,VaushV,NaN,Would be a shame if this unofficial site received a lot of bad data,1668471465,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data
2,2,VaushV,NaN,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,1668471209,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.
3,3,VaushV,"Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)",How do you feel about the global population reaching 8 billion?,1668470313,"How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\nView Poll","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. View Poll"
4,4,VaushV,"Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trump being held accountable or Stopping the rise o...",Hot take: Trump SHOULDN’T be arrested/indicted,1668468418,"Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trum...","Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secu

In [20]:
vec_vd_2 = pd.concat([vec_df,vaush_destiny_df], axis=1, ignore_index=True)
vec_vd_2.columns = list(vec_df.columns)+list(vaush_destiny_df.columns)

In [21]:
vec_vd_2

,00,00 best,00 eastern,00 eventually,00 mark,00 maybe,00 meaningless,00 onwards,000,000 000,...,이우찬,이우찬 actually,index,subreddit,selftext,title,created_utc,full_text,no_links,no_links_2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,VaushV,"Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?",Anyone Noticing Vaush's Memory Issues Lately?,1668472458,"Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?","Anyone Noticing Vaush's Memory Issues Lately? Seems like every time he makes an argument he says ""there was a study, I forget... it found something like... I'll have to read up on it again."" You know what D does this too. Can these streamers admit they don't have the time to learn things or are we supposed to believe they ACTUALLY read their sources cited. Or is this a rhetorical tactique?"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,VaushV,NaN,Would be a shame if this unofficial site received a lot of bad data,1668471465,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data,Would be a shame if this unofficial site received a lot of bad data
2,0,0,0,0,0,0,0,0,0,0,...,0,0,2,VaushV,NaN,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,1668471209,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.,🏳️‍⚧️🏳️‍⚧️Trans colored boat at Disney World! Ron DeSantis would not approve.
3,0,0,0,0,0,0,0,0,0,0,...,0,0,3,VaushV,"Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)",How do you feel about the global population reaching 8 billion?,1668470313,"How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\n[View Poll](https://www.reddit.com/poll/yvgwwx)","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. \n\nView Poll","How do you feel about the global population reaching 8 billion? Personally, IDK. It just seems like too many people lol. View Poll"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,4,VaushV,"Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think Desantis is infinitely more dangerous than Trump \n\nSo what’s more important? Trump being held accountable or Stopping the rise o...",Hot take: Trump SHOULDN’T be arrested/indicted,1668468418,"Hot take: Trump SHOULDN’T be arrested/indicted Right now, if Trump and Biden went for a rematch, Biden would win\n\nBut if Trump gets arrested and is unable to secure the republican nomination, we KNOW Desantis is up next\n\nAnd Every(and I mean EVERY) poll shows Desantis beating Biden\n\nMe personally, I think D

In [22]:
vec_vd_2.columns

Index(['00', '00 best', '00 eastern', '00 eventually', '00 mark', '00 maybe',
       '00 meaningless', '00 onwards', '000', '000 000',
       ...
       '이우찬', '이우찬 actually', 'index', 'subreddit', 'selftext', 'title',
       'created_utc', 'full_text', 'no_links', 'no_links_2'],
      dtype='object', length=219530)

In [ ]:
sub_df = vec_vd_2.drop(columns = ['selftext', 'title',
       'created_utc', 'full_text', 'no_links', 'no_links_2'])

In [ ]:
vaush_words = sub_df.loc[sub_df]